In [34]:
pip install tensorflowjs

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, layers, models, regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.lite as tflite
import tensorflowjs as tfjs

In [36]:
!pip freeze > req.txt

In [37]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.3.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.13
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.7.1
arviz                              0.20.0
astropy                            7.0.1
astropy-iers-data                  0.2025.3.10.0.29.26
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              25.3.0
audioread          

In [38]:
!pip check

ipython 7.34.0 requires jedi, which is not installed.
pygobject 3.42.1 requires pycairo, which is not installed.
torch 2.6.0+cu124 has requirement nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2.
torch 2.6.0+cu124 has requirement nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82.
torch 2.6.0+cu124 has requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82.
torch 2.6.0+cu124 has requirement nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82.
torch 2.6.0+cu124 has requirement nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75.
torch 2.6.0+cu124 has require

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
img_size = (128,128)
batch_size =  32

train_dataset = image_dataset_from_directory(
    "drive/MyDrive/Colab_Notebooks/archive_23/train",
    image_size=img_size,
    batch_size=batch_size,
)

valid_dataset = image_dataset_from_directory(
    "drive/MyDrive/Colab_Notebooks/archive_23/valid",
    image_size=img_size,
    batch_size=batch_size,
)

test_dataset = image_dataset_from_directory(
    "drive/MyDrive/Colab_Notebooks/archive_23/test",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_dataset.class_names
print("Class Names:", class_names)

Found 7624 files belonging to 53 classes.
Found 265 files belonging to 53 classes.
Found 265 files belonging to 53 classes.
Class Names: ['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of h

In [41]:
AUTOTUNE = tf.data.AUTOTUNE

normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
valid_dataset = valid_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# data_augmentation = tf.keras.Sequential([
#     tf.keras.layers.RandomFlip("horizontal"),
#     tf.keras.layers.RandomRotation(0.05),
#     tf.keras.layers.RandomZoom(0.05),
# ])

# # Apply augmentation only to the training set
# def augment(image, label):
#     return data_augmentation(image), label

# train_dataset = train_dataset.map(augment)

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [42]:
model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(32, (3,3), padding="same", kernel_regularizer=regularizers.l2(0.001 ), input_shape=(128, 128, 3)),
    BatchNormalization(),  # Place before activation
    layers.Activation("relu"),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),

    # Second Convolutional Block
    layers.Conv2D(64, (3,3), padding="same", kernel_regularizer=regularizers.l2(0.001 )),
    BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),

    # Third Convolutional Block
    layers.Conv2D(128, (3,3), padding="same", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    # Fourth Convolutional Block
    layers.Conv2D(256, (3,3), padding="same", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    # Flatten and Fully Connected Layers
    layers.Flatten(),
    layers.Dense(384, kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
    BatchNormalization(),
    layers.Activation("relu"),
    layers.Dropout(0.5),

    layers.Dense(len(class_names), activation="softmax")  # Output layer
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])

# Display model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_15 (Activation)           │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_16 (Activation)           │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 32, 32, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_17 (Activation)           │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 16, 16, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 16, 16, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_18 (Activation)           │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 6,704,117 (25.57 MB)

 Trainable params: 6,702,389 (25.57 MB)

 Non-trainable params: 1,728 (6.75 KB)

In [43]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, min_lr=1e-6)

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

model_checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True)

In [44]:
def graph():
    # Extract accuracy and loss values
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(1, len(train_loss) + 1)

    # Plot Loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()

    plt.show()

In [45]:
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=25,
    callbacks=[reduce_lr,early_stop,model_checkpoint] #early_stop
)

Epoch 1/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1734 - loss: 4.6333

239/239 ━━━━━━━━━━━━━━━━━━━━ 556s 2s/step - accuracy: 0.1737 - loss: 4.6314 - val_accuracy: 0.0415 - val_loss: 5.7192 - learning_rate: 0.0010
Epoch 2/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3923 - loss: 3.5619

239/239 ━━━━━━━━━━━━━━━━━━━━ 544s 2s/step - accuracy: 0.3925 - loss: 3.5615 - val_accuracy: 0.5132 - val_loss: 3.1915 - learning_rate: 0.0010
Epoch 3/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5078 - loss: 3.1758

239/239 ━━━━━━━━━━━━━━━━━━━━ 544s 2s/step - accuracy: 0.5080 - loss: 3.1755 - val_accuracy: 0.6906 - val_loss: 2.6329 - learning_rate: 0.0010
Epoch 4/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6335 - loss: 2.9104

239/239 ━━━━━━━━━━━━━━━━━━━━ 541s 2s/step - accuracy: 0.6336 - loss: 2.9101 - val_accuracy: 0.7585 - val_loss: 2.5351 - learning_rate: 0.0010
Epoch 5/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 528s 2s/step - accuracy: 0.6987 - loss: 2.7578 - val_accuracy: 0.7547 - val_loss: 2.6493 - learning_rate: 0.0010
Epoch 6/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7436 - loss: 2.7204

...

239/239 ━━━━━━━━━━━━━━━━━━━━ 561s 2s/step - accuracy: 0.8596 - loss: 2.7892 - val_accuracy: 0.8981 - val_loss: 2.6791 - learning_rate: 0.0010
Epoch 17/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 562s 2s/step - accuracy: 0.8661 - loss: 2.7579 - val_accuracy: 0.8943 - val_loss: 2.6296 - learning_rate: 0.0010
Epoch 18/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8633 - loss: 2.7595

239/239 ━━━━━━━━━━━━━━━━━━━━ 560s 2s/step - accuracy: 0.8633 - loss: 2.7595 - val_accuracy: 0.9132 - val_loss: 2.6275 - learning_rate: 0.0010
Epoch 19/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 531s 2s/step - accuracy: 0.8652 - loss: 2.7395 - val_accuracy: 0.9019 - val_loss: 2.6170 - learning_rate: 0.0010
Epoch 20/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 538s 2s/step - accuracy: 0.8597 - loss: 2.7815 - val_accuracy: 0.9094 - val_loss: 2.5957 - learning_rate: 0.0010
Epoch 21/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8740 - loss: 2.6829

239/239 ━━━━━━━━━━━━━━━━━━━━ 534s 2s/step - accuracy: 0.8739 - loss: 2.6829 - val_accuracy: 0.9245 - val_loss: 2.5946 - learning_rate: 0.0010
Epoch 22/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8725 - loss: 2.7309

239/239 ━━━━━━━━━━━━━━━━━━━━ 543s 2s/step - accuracy: 0.8724 - loss: 2.7309 - val_accuracy: 0.9283 - val_loss: 2.5941 - learning_rate: 0.0010
Epoch 23/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 532s 2s/step - accuracy: 0.8799 - loss: 2.6729 - val_accuracy: 0.9019 - val_loss: 2.5373 - learning_rate: 0.0010
Epoch 24/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8740 - loss: 2.6736

239/239 ━━━━━━━━━━━━━━━━━━━━ 533s 2s/step - accuracy: 0.8740 - loss: 2.6736 - val_accuracy: 0.9509 - val_loss: 2.4666 - learning_rate: 0.0010
Epoch 25/25
239/239 ━━━━━━━━━━━━━━━━━━━━ 563s 2s/step - accuracy: 0.8924 - loss: 2.6157 - val_accuracy: 0.9245 - val_loss: 2.5132 - learning_rate: 0.0010


In [46]:
test_loss, test_acc = model.evaluate(test_dataset)
print("Test Accuracy:", test_acc)

9/9 ━━━━━━━━━━━━━━━━━━━━ 37s 4s/step - accuracy: 0.9320 - loss: 2.4722
Test Accuracy: 0.9207547307014465


In [ ]:
graph()

In [48]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF-Lite model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


tfjs.converters.save_keras_model(model, "tfjs_model")

Saved artifact at '/tmp/tmpyeos77e1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_81')
Output Type:
  TensorSpec(shape=(None, 53), dtype=tf.float32, name=None)
Captures:
  137479175553488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137479175554640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137479175553296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137479175555984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137479175554448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137479175554832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137479175555216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137479175557904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137481293744656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137481293745040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1374812937

failed to lookup keras version from the file,
    this is likely a weight only file
